In [1]:
!pwd

/home/patel.devki1/Dataset


In [8]:
import os
import re

# Directory where the images are stored
directory = '/home/patel.devki1/Dataset/lung_image_sets/lung_n/lung_n_'

# Regex pattern to match files like 'lungn1.jpeg'
pattern = re.compile(r'lungn(\d+)\.jpeg')

# List all files in the directory and filter using the regex pattern
files = os.listdir(directory)
lungn_files = [f for f in files if pattern.match(f)]

# Count the number of images
total_images = len(lungn_files)
print("Total number of images:", total_images)


Total number of images: 5000


In [9]:
import os

# Directory where the images are stored
directory = '/home/patel.devki1/Dataset/lung_image_sets/lung_n/lung_n_'

# Get all files in the directory
files = os.listdir(directory)

# Extract the numerical part from each file name and convert it to integers
actual_numbers = [int(file.split('.')[0].split('lungn')[-1]) for file in files if file.startswith('lungn')]

# Generate a list of expected numerical file names based on the range of numbers in the actual files
expected_numbers = list(range(min(actual_numbers), max(actual_numbers) + 1))

# Find the missing file numbers by subtracting the actual numbers from the expected numbers
missing_numbers = set(expected_numbers) - set(actual_numbers)

# Print the missing file numbers
if missing_numbers:
    print("Missing files:")
    for num in missing_numbers:
        print(f"lungn{num}.jpeg")
else:
    print("No missing files found.")


No missing files found.


In [4]:
import os
import re

# Directory where the images are stored
directory = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc'

# Regex pattern to match files like 'lungn1.jpeg'
pattern = re.compile(r'lungscc(\d+)\.jpeg')

# List all files in the directory and filter using the regex pattern
files = os.listdir(directory)
lungscc_files = [f for f in files if pattern.match(f)]

# Count the number of images
total_images = len(lungscc_files)
print("Total number of images:", total_images)


Total number of images: 0


In [5]:
import os

# Directory where the images are stored
directory = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc/lung_scc_'

# Get all files in the directory
files = os.listdir(directory)

# Extract the numerical part from each file name and convert it to integers
actual_numbers = [int(file.split('.')[0].split('lungscc')[-1]) for file in files if file.startswith('lungscc')]

# Generate a list of expected numerical file names based on the range of numbers in the actual files
expected_numbers = list(range(min(actual_numbers), max(actual_numbers) + 1))

# Find the missing file numbers by subtracting the actual numbers from the expected numbers
missing_numbers = set(expected_numbers) - set(actual_numbers)

# Print the missing file numbers
if missing_numbers:
    print("Missing files:")
    for num in missing_numbers:
        print(f"lungscc{num}.jpeg")
else:
    print("No missing files found.")


No missing files found.


In [6]:
pip install torch torchvision


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Define the path to your original dataset directory
original_data_dir = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc/lung_scc_'

# Define paths for train and test directories
# train_dir = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc/train_data/'
# test_dir = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc/test_data/'

train_dir = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc/lung_scc_train_data'
test_dir = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc/lung_scc_test_data'

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get the list of all files in the original dataset directory
all_files = os.listdir(original_data_dir)

# Split the dataset into train and test sets (80-20 split)
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

# Move files to train directory
for file in train_files:
    src = os.path.join(original_data_dir, file)
    dst = os.path.join(train_dir, file)
    if os.path.isdir(src):
        if not os.path.exists(dst):  # Check if destination directory already exists
            shutil.copytree(src, dst)  # Copy entire directory and its contents
    else:
        if not os.path.exists(dst):  # Check if file already exists in destination
            shutil.copyfile(src, dst)  # Copy individual file if it doesn't exist

# Move files to test directory
for file in test_files:
    src = os.path.join(original_data_dir, file)
    dst = os.path.join(test_dir, file)
    if os.path.isdir(src):
        if not os.path.exists(dst):  # Check if destination directory already exists
            shutil.copytree(src, dst)  # Copy entire directory and its contents
    else:
        if not os.path.exists(dst):  # Check if file already exists in destination
            shutil.copyfile(src, dst)  # Copy individual file if it doesn't exist


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

train_data_new = os.path.join(train_dir, 'train')
test_data_new = os.path.join(test_dir, 'test')

# Create datasets from folders
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [12]:
# Define the CNN model
class LungNet(nn.Module):
    def __init__(self, num_classes=3):
        super(LungNet, self).__init__()
        # Define your model architecture here
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = LungNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 100.00%


In [13]:
import os

# Define the path to the folder you want to check permissions for
directory = '/home/patel.devki1/Dataset/lung_image_sets/lung_scc'

folder_path = directory

# Use the os.access() function to check permissions
readable = os.access(folder_path, os.R_OK)
writable = os.access(folder_path, os.W_OK)
executable = os.access(folder_path, os.X_OK)

# Print the permissions status
print(f'Folder is readable: {readable}')
print(f'Folder is writable: {writable}')
print(f'Folder is executable: {executable}')


Folder is readable: True
Folder is writable: True
Folder is executable: True


In [36]:
!pip show torch

Name: torch
Version: 2.2.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/patel.devki1/.local/lib/python3.8/site-packages
Requires: networkx, fsspec, nvidia-cublas-cu12, nvidia-cudnn-cu12, nvidia-curand-cu12, sympy, nvidia-nccl-cu12, nvidia-cufft-cu12, nvidia-cuda-nvrtc-cu12, nvidia-nvtx-cu12, filelock, nvidia-cusparse-cu12, triton, nvidia-cuda-runtime-cu12, nvidia-cuda-cupti-cu12, jinja2, typing-extensions, nvidia-cusolver-cu12
Required-by: torchvision


In [37]:
!pip show torchvision

Name: torchvision
Version: 0.17.2
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /home/patel.devki1/.local/lib/python3.8/site-packages
Requires: torch, numpy, pillow
Required-by: 
